# TODO

- vector_store.ipynb
    - data/소설들.pdf 들 Vector store에 저장.
        - chroma, pinecone (알아서)
        - metadata 
            - title: 소설제목
            - author: 저자
            - full_text: 소설 전체 내용
        - Document(page_content="split된 내용", metadata={})
        - page_content는 조회할 때 사용할 embedding vector를 생성할 때 사용될 내용.
        - llm 모델에 전송할 context는 metadata의 full_text

- app.py
    - streamlit을 이용해서 서비스하는 application
    - vector_store.ipynb에서 구축한 DB를 이용해 질문과 관련된 내용들을 
      조회해서 llm에 전송하고 그 결과를 채팅창에 출력.
    - memory 기능은 사용하지 않는다.

# 소설들을 Vector Store에 저장

## Load
- `data/*.pdf` 파일들을 모두 load

- 각 소설들을 split 한 뒤 vector store에 저장한다.   
- metadata:
    - title: 소설제목
    - author: 저자
    - full_text: 소설 전체 내용

In [ ]:
import re
import os
import config
from glob import glob

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv

load_dotenv()

# 설정 정보 읽기
CHUNK_SIZE = config.chunk_size
CHUNK_OVERLAP = config.chunk_overlap

MODEL_NAME  = config.model_name
EMBEDDING_NAME = config.embedding_name

COLLECTION_NAME = config.collection_name
PERSIST_DIRECTORY = config.persist_directory

# 소설 파일들 경로 조회
path_list = glob("data/*.pdf")

document_list = []

# load -> 전처리 -> split -> metadata 추가 
for path in path_list:
    # Document Load
    loader = PyMuPDFLoader(path)
    load_docs = loader.load()

    # 전처리 - 페이지 번호 제거, 🙝🙟 를 \n\n 으로 변경
    full_text = [doc.page_content for doc in load_docs]
    full_text = ''.join(full_text)
    full_text = full_text.replace("🙝🙟", "\n\n")
    full_text = re.sub(r"\d+\n", " ", full_text)   # 페이지 번호 제거
    
    # Split
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name=MODEL_NAME,
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
    )
    docs = splitter.split_text(full_text)

    # Metadata 생성
    author = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
    metadata = {
        "title":load_docs[0].metadata["title"],
        "author": author,
        "full_text":full_text
    }

    # Document 생성
    for doc in docs:
        _doc = Document(metadata=metadata, page_content=doc)
        document_list.append(_doc)

print(len(document_list))

## Vector Store 저장
- Chroma Vector Store에 저장

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)

vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# 확인

In [ ]:
# 자정 개수 확인
vector_store._collection.count()

In [10]:
###########################
# 검색

query = "현진건의 소설들의 특징은 어떤 것이 있을까?" 
query = "운수좋은 날에서 주인공은 왜 괴상하게도 운수가 좋다고 느꼈을까?"
query = "허 생원은 어느 장으로 가서 장사를 할 예정인가?"
query = "이효석의 메밀꽃 필 무렵의 주인공은 누구인가?"
query = "벙어리 삼룡이는 화자가 몇살 때 일인가?"
query = "삼룡이의 주인 아들의 색시는 몇살인가?"

result = vector_store.similarity_search(
    query,
    k=5,
    # filter={"author": "현진건"}
)

In [ ]:
for res in result:
    print(res.metadata['title'], res.metadata['author'], sep=" | ")
    print(res.page_content)
    print("=====================================")